In [1]:
pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 3.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [2]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 37.9 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [3]:
!pip install -i https://pypi.org/simple/ bitsandbytes

Looking in indexes: https://pypi.org/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 12.8 MB/s eta 0:00:00


In [4]:
!pip install -U "transformers==4.38.1" --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 8.6 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.40.0
    Uninstalling transformers-4.40.0:
      Successfully uninstalled transformers-4.40.0


In [6]:
# Imports
import json
import re
import os
import numpy as np
import pandas as pd
from datasets import load_dataset

from google.colab import userdata
access_token = userdata.get('HF_TOKEN')

from transformers import AutoTokenizer, pipeline
import torch

# Dataset

In [7]:
#load the data for all languages
data = load_dataset("Eloquent/HalluciGen-Translation", "test_generation")

print(data.keys())

data_de_en = data['test_generation_de_en'].to_pandas()
data_en_de = data['test_generation_en_de'].to_pandas()
data_fr_en = data['test_generation_fr_en'].to_pandas()
data_en_fr = data['test_generation_en_fr'].to_pandas()

display(data_de_en.head())
display(data_en_de.head())

Generating test_generation_de_en split:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating test_generation_en_de split:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating test_generation_fr_en split:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating test_generation_en_fr split:   0%|          | 0/100 [00:00<?, ? examples/s]

dict_keys(['test_generation_de_en', 'test_generation_en_de', 'test_generation_fr_en', 'test_generation_en_fr'])


,id,langpair,source
0,0,de-en,In Ländern ist das Anbieten von Luxus-Gästehäu...
1,1,de-en,"Dies ist in Großbritannien gängige Praxis, abe..."
2,2,de-en,„Auf Grundlage dieses Fossils lässt sich sagen...
3,3,de-en,"Wir werden niemals ein Auto überholen, einen B..."
4,4,de-en,"Ein Minenabwehrschiff der Avenger-Klasse, das ..."


,id,langpair,source
0,0,en-de,He graduated from the College of Arts & Scienc...
1,1,en-de,"Most are independent nations, or associated wi..."
2,2,en-de,But there are a lot of things about birds that...
3,3,en-de,The police had trouble using their speed radar...
4,4,en-de,"""The Giza Plateau, or """"Giza Necropolis"""" in t..."


## Load Gamma model

In [8]:
#define the Model
model = "google/gemma-7b-it"
# use quantized model
pipeline = pipeline(
    "text-generation",
    model=model,
    model_kwargs={
        "torch_dtype": torch.float16,
        "quantization_config": {"load_in_4bit": True}
    },
)

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

## Generate Data

In [21]:
def generate_hypothesis(source, target_language):

  answer_format = {"hyp+": "", "hyp-": ""}

  # prompt for translation generation task
  user_prompt = f'''You are a text generator and your task is to generate two translation hypothesis given the 'src' below.
      The first translation labelled as 'hyp+' should be supported by 'src' and the second translation labelled as 'hyp-' should not be supported by 'src'.
      Provide the result in the following format: {answer_format}. Target language: {target_language}

       Src: {source}

       Result:
       '''


  messages = [
    {"role": "user", "content": user_prompt},
  ]

  prompt = pipeline.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

  terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

  outputs = pipeline(
    prompt,
    max_new_tokens=200,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    top_p=0.95
  )

  #print("Gen text: ", outputs[0]["generated_text"][len(prompt):])
  generated_translation = outputs[0]["generated_text"][len(prompt):]
  #print(generated_translation)

  return generated_translation

## Save Data as csv file

In [10]:
def extract_json_data(result_text):
    # Find the start and end indices of the JSON data
    start_index = result_text.find('{')
    end_index = result_text.find('}') + 1  # Add 1 to include the closing brace

    # Extract the JSON data from the string
    result_text = result_text[start_index:end_index]
    #print("result text: ", result_text)

    # Replace single quotes used as delimiters with double quotes
    result_text = json.loads(result_text.replace("'", "\""))
    return result_text

In [ ]:
for x in data_de_en['source'][0:3]:
  result = generate_hypothesis(x, "English")
  extract_json_data(result)

/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


{'hyp+': 'In countries, the offering of luxury guesthouses has been elevated to an art form today.', 'hyp-': 'In countries, the offering of luxury guesthouses has been elevated to a new art form today.'}

The source text states that the offering of luxury guesthouses has been elevated to an art form in countries. Therefore, the translation hypothesis hyp+ is supported by the source text. The translation hypothesis hyp- is not supported by the source text as it states that the offering of luxury guesthouses has been elevated to a new art form.
{'hyp+': 'The practice of releasing photos is commonplace in the UK, but the Scottish justice system operates differently, and the courts have expressed concerns about the potential harm of publishing photos.', 'hyp-': 'The practice of releasing photos is a common occurrence in the UK, but the Scottish justice system operates similarly, and the courts have expressed concerns about the potential harm of publishing photos.'}
{'hyp+': "Based on this 

In [11]:
#save results
def save_results(target_language, df, csv_filename):
    skipped_rows = []
    # Split DataFrame into chunks of 10 rows
    chunks = np.array_split(df, len(df) // 10 if len(df) % 10 == 0 else len(df) // 10 + 1)

    for chunk in chunks:
        # Iterate through each row in the chunk
        for index, row in chunk.iterrows():
            source = row['source']

            try:
                # Generate translation
                result = generate_hypothesis(source, target_language)
                extracted_hyp = extract_json_data(result)

                hyp_support = extracted_hyp['hyp+']
                #print(hyp_support)
                hyp_hallu = extracted_hyp['hyp-']
                #print(hyp_hallu)
                #print('source:',source)

                # Update DataFrame with new values
                chunk.at[index, 'hyp+'] = hyp_support
                chunk.at[index, 'hyp-'] = hyp_hallu
                #chunk.at[index, 'prompt'] = prompt
            except:
                print(f"Skipping row {index} due to error in data extraction.")
                skipped_rows.append(index)
                continue

        # Append chunk to CSV file
        if not os.path.isfile(csv_filename):
            chunk.to_csv(csv_filename, mode='w', header=True, index=False)
        else:
            chunk.to_csv(csv_filename, mode='a', header=False, index=False)

        # Load CSV file
        df = pd.read_csv(csv_filename)
        display(df)
    return skipped_rows

In [12]:
def fix_skipped_rows(target_language, df, csv_filename, skipped_rows):
    skipped_rows2 = []
    for index in skipped_rows:
        row = df.loc[index]

        source = row['source']

        try:
            result = generate_hypothesis(source, target_language)
            # Find the start and end indices of the JSON data
            start_index = result.find('{')
            end_index = result.find('}') + 1
            result_text = result[start_index:end_index]
            result_text = result_text.replace('"', "'").replace("{'hyp+': '", '{"hyp+": "').replace(".', 'hyp-': '", '.", "hyp-": "').replace(".',\n'hyp-': '", '.",\n"hyp-": "').replace(".', \n'hyp-': '", '.", \n"hyp-": "').replace("'}", '"}')
            print("result text ", result_text)
            #print("result text ", result_text)
            extracted_hyp = json.loads(result_text)
            df.at[index, 'hyp+'] =  extracted_hyp['hyp+']
            df.at[index, 'hyp-'] =  extracted_hyp['hyp-']
        except Exception as e:
            print(f"Skipping row {index} due to error in data extraction.")
            skipped_rows2.append(index)
            continue

    df.to_csv(csv_filename, index=False)
    return skipped_rows2

In [13]:
def fix_skipped_rows_manual(target_language, df, skipped_rows):
    for index in skipped_rows:
        row = df.loc[index]

        source = row['source']

        result = generate_hypothesis(source, target_language)
        # Find the start and end indices of the JSON data
        start_index = result.find('{')
        end_index = result.find('}') + 1
        result_text = result[start_index:end_index]
        #result_text = result_text.replace('"', "'").replace("{'hyp+': '", '{"hyp+": "').replace(".', 'hyp-': '", '.", "hyp-": "').replace(".',\n'hyp-': '", '.",\n"hyp-": "').replace(".', \n'hyp-': '", '.", \n"hyp-": "').replace("'}", '"}')
        print("result text ", result_text)
        #print("result text ", result_text)
        #extracted_hyp = json.loads(result_text)
        #df.at[index, 'hyp+'] =  extracted_hyp['hyp+']
        #df.at[index, 'hyp-'] =  extracted_hyp['hyp-']


    #df.to_csv(csv_filename, index=False)

# Translation German - English

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/gemma/results_gemma_de_en.csv"

skipped_rows_de_en = save_results("English", data_de_en, csv_filename)

/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Skipping row 3 due to error in data extraction.
Skipping row 6 due to error in data extraction.


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Skipping row 26 due to error in data extraction.
Skipping row 28 due to error in data extraction.


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


...

Skipping row 73 due to error in data extraction.


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Skipping row 84 due to error in data extraction.
Skipping row 85 due to error in data extraction.
Skipping row 86 due to error in data extraction.
Skipping row 87 due to error in data extraction.


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Skipping row 91 due to error in data extraction.


In [ ]:
print(skipped_rows_de_en)
print(len(skipped_rows_de_en))

[3, 6, 26, 28, 31, 34, 41, 44, 46, 63, 64, 73, 84, 85, 86, 87, 91]
17


In [15]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/gemma/results_gemma_de_en_fixed.csv"
df_results_de_en = pd.read_csv(csv_filename)
len(df_results_de_en)

100

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/gemma/results_gemma_de_en_fixed.csv"
skipped_rows_de_en = fix_skipped_rows("English", df_results_de_en, csv_filename, skipped_rows_de_en)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


result text  {"hyp+": "We will never be overtaken by a car, build a fountain in our backyard, travel to investigate ancient ruins, or remodel our neighbor's house.", "hyp-": "We will never be able to afford a car, build a fountain in our backyard, travel to investigate ancient ruins, or remodel our neighbor's house."}
result text  {"hyp+": "Cold weather might be the only danger that could overcome unprepared people.", "hyp-": "Cold weather is probably the only danger that could overcome unprepared people."}
result text  {"hyp+": "The worker drills a hole for the secretary.", "hyp-": "The worker is digging a hole for the secretary."}
result text  {"hyp+": "Cardinal Beaton was Lord Chancellor of Scotland, Archbishop of St. Andrews, and Legate in Scotland at the time of his death.", "hyp-": "Cardinal Beaton was a powerful figure in Scotland at the time of his death."}
result text  {"hyp+": "Coolidge played the dramatic role of 'Geneviève McDonagh' in 2009.", "hyp-": "Coolidge played the r

In [ ]:
print(skipped_rows_de_en)
print(len(skipped_rows_de_en))

[63, 64]
2


In [16]:
fix_skipped_rows_manual("English", df_results_de_en, skipped_rows_de_en)

NameError: name 'skipped_rows_de_en' is not defined

In [17]:
result = generate_hypothesis(df_results_de_en['source'][64], "English")
print(result)

/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


{'hyp+': 'The territorial area of Turkey, including its lakes, covers 783.562 square kilometers (300.948 square miles), of which 755.688 square kilometers (291.773 square miles) are located in Southwest Asia and 23.764 square kilometers (9.174 square miles) in Europe.', 'hyp-': 'The territorial area of Turkey, including its lakes, covers 783.562 square kilometers (300.948 square miles), of which 755.688 square kilometers (291.773 square miles) are located in


# Translation English - German

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/gemma/results_gemma_en_de.csv"

skipped_rows_en_de = save_results("German", data_en_de, csv_filename)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Skipping row 4 due to error in data extraction.
Skipping row 5 due to error in data extraction.


,id,langpair,source,hyp+,hyp-
0,0,en-de,He graduated from the College of Arts & Scienc...,Er schloss sein Studium an der Kunst- und Wiss...,Er schloss sein Studium an der Kunst- und Wiss...
1,1,en-de,"Most are independent nations, or associated wi...",Die meisten Nationen sind unabhängige Nationen...,Die meisten Nationen sind unabhängige Nationen...
2,2,en-de,But there are a lot of things about birds that...,"Es gibt viele Dinge bei Vögeln, die wie ein Di...","Es gibt viele Dinge bei Vögeln, die wie ein Di..."
3,3,en-de,The police had trouble using their speed radar...,"Die Polizei hatte Probleme, ihre Geschwindigke...","Die Polizei hatte Probleme, ihre Geschwindigke..."
4,4,en-de,"""The Giza Plateau, or """"Giza Necropolis"""" in t...",NaN,NaN
5,5,en-de,Prime Minister Stephen Harper has agreed to se...,NaN,NaN
6,6,en-de,The temple of Artemis was destroyed on July 21...,Das Tempel der Artemis wurde am 21. Juli 356 v...,Das Tempel der Artemis wurde am 21. Juli 356 v...
7,7,en-de,South Africa have defeated the All Blacks (New...,Südafrika hat die All Blacks (Neuseeland) in e...,Südafrika hat die All Blacks (Neuseeland) in e...
8,8,en-de,"condition is stable, though he will be isolat...","Die Situation ist stabil, obwohl er infolge de...","Die Situation ist stabil, obwohl er infolge de..."
9,9,en-de,"The world has over 5,000 different languages, ...","Die Welt hat über 5.000 verschiedene Sprachen,...",Die Welt hat mehr als 5.000 verschiedene Sprac...


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Skipping row 10 due to error in data extraction.


...

Skipping row 87 due to error in data extraction.
Skipping row 89 due to error in data extraction.


,id,langpair,source,hyp+,hyp-
0,0,en-de,He graduated from the College of Arts & Scienc...,Er schloss sein Studium an der Kunst- und Wiss...,Er schloss sein Studium an der Kunst- und Wiss...
1,1,en-de,"Most are independent nations, or associated wi...",Die meisten Nationen sind unabhängige Nationen...,Die meisten Nationen sind unabhängige Nationen...
2,2,en-de,But there are a lot of things about birds that...,"Es gibt viele Dinge bei Vögeln, die wie ein Di...","Es gibt viele Dinge bei Vögeln, die wie ein Di..."
3,3,en-de,The police had trouble using their speed radar...,"Die Polizei hatte Probleme, ihre Geschwindigke...","Die Polizei hatte Probleme, ihre Geschwindigke..."
4,4,en-de,"""The Giza Plateau, or """"Giza Necropolis"""" in t...",NaN,NaN
...,...,...,...,...,...
85,85,en-de,The NSA has its own internal data format that ...,"Die NSA verfügt über ein internes Datenformat,...","Die NSA verfügt über ein internes Datenformat,..."
86,86,en-de,"It was in the early days of GPS, the car knew ...","In den frühen Tagen von GPS, wusste das Auto, ...","In den frühen Tagen von GPS, wusste das Auto, ..."
87,87,en-de,He could ask that because he knew New Yorkers ...,NaN,NaN
88,88,en-de,"Information is powerful, but it is how we use ...","Die Informationen sind mächtig, aber es ist, w...","Die Informationen sind mächtig, aber es ist, w..."


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Skipping row 94 due to error in data extraction.
Skipping row 95 due to error in data extraction.
Skipping row 98 due to error in data extraction.


,id,langpair,source,hyp+,hyp-
0,0,en-de,He graduated from the College of Arts & Scienc...,Er schloss sein Studium an der Kunst- und Wiss...,Er schloss sein Studium an der Kunst- und Wiss...
1,1,en-de,"Most are independent nations, or associated wi...",Die meisten Nationen sind unabhängige Nationen...,Die meisten Nationen sind unabhängige Nationen...
2,2,en-de,But there are a lot of things about birds that...,"Es gibt viele Dinge bei Vögeln, die wie ein Di...","Es gibt viele Dinge bei Vögeln, die wie ein Di..."
3,3,en-de,The police had trouble using their speed radar...,"Die Polizei hatte Probleme, ihre Geschwindigke...","Die Polizei hatte Probleme, ihre Geschwindigke..."
4,4,en-de,"""The Giza Plateau, or """"Giza Necropolis"""" in t...",NaN,NaN
...,...,...,...,...,...
95,95,en-de,"If you invent something, Tesla invented electr...",NaN,NaN
96,96,en-de,"LP: The amazing thing about this is this is, I...",The amazing thing about this is that the syste...,The amazing thing about this is that the syste...
97,97,en-de,"LP: The amazing thing about this is this is, I...","The system recognizes the pixels, controls, an...",The system recognizes the pixels and controls ...
98,98,en-de,"""She said, """"I used to have a job but I had to...",NaN,NaN


In [ ]:
print(skipped_rows_en_de)
print(len(skipped_rows_en_de))

[4, 5, 10, 17, 18, 21, 23, 27, 33, 45, 49, 57, 59, 62, 69, 75, 87, 89, 94, 95, 98]
21


In [18]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/gemma/results_gemma_en_de_fixed.csv"
df_results_en_de = pd.read_csv(csv_filename)
len(df_results_en_de)

100

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/gemma/results_gemma_en_de_fixed.csv"
skipped_rows_en_de = fix_skipped_rows("German", df_results_en_de, csv_filename, skipped_rows_en_de)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


result text  {"hyp+": "Die Giza-Plattform, auch 'Giza Necropolis'' in der ägyptischen Valley of the Dead beinhaltet mehrere Pyramide (von denen die Pyramide die größte ist), mehrere kleine Tomten, mehrere Tempeln und die große Sphinx.", "hyp-": "Die Giza-Plattform, auch 'Giza Necropolis'' in der ägyptischen Valley of the Dead beinhaltet mehrere Pyramide, mehrere kleine Tomten, mehrere Tempeln und die große Sphinx. Es beinhaltet auch eine große Menge anderer Gebäude."}
result text  
Skipping row 5 due to error in data extraction.
result text  {"hyp+": "Arthur Guinness unterschrieb am 24. September 1759 einen 9.000-Jahres-Pachtvertrag für die St. James' Gate-Brauerei in Dublin, Irland.", "hyp-": "Arthur Guinness unterschrieb am 24. September 1759 einen Pachtvertrag für die St. James' Gate-Brauerei in Dublin, Irland."}
result text  {"hyp+": "The Onion has become a successful news parody empire with a large following.", "hyp-": "The Onion has become a powerful news organization with a sign

In [ ]:
print(skipped_rows_en_de)

[5, 33, 49, 59, 62, 75, 87, 89, 94, 95, 98]


In [ ]:
skipped_rows_en_de = [5, 33, 49, 59, 62, 75, 87, 89, 94, 95, 98]
fix_skipped_rows_manual("German", df_results_en_de, skipped_rows_en_de)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


result text  
result text  {'hyp+': 'Es war von Willard Mack inszeniert und basiert sich auf einem Spiel aus dem Jahr 1917, ``Tiger Rose '', von George Fitzmaurice.', 'hyp-' : 'Es war von Willard Mack inszeniert und basiert sich auf einem Spiel aus dem Jahr 1917, ``Tiger Rose '', von George Fitzmaurice. Es war ein Spiel, das nach der Premiere große Aufmerksamkeit erhalten hat.'}
result text  {'hyp+': 'Simon Simon wurde in Charlie und Megan Pace in 1979 geboren und lebte in Manchester, England.', 'hyp-': 'Simon Simon war in Charlie und Megan Pace geboren und lebte in Manchester, England.'}
result text  
result text  {'hyp+': 'The word "Gypsy" is often considered offensive because of its associations with negative stereotypes and inaccurate perceptions of Romani people.', 'hyp-': 'The word "Gypsy" is often considered offensive because it is a derogatory term used against Romani people.'}
result text  {'hyp+': 'Er verließ sie und danach mit Wolverstone an der Schiene, beobachtete er den H

In [22]:
result = generate_hypothesis(df_results_en_de['source'][59], "German")
print(result)

{'hyp+': "Es beginnt fünf Minuten in der Darstellung ein Wind, der beginnt zu rollen, nachdem ungefähr eine Minute, erreicht der Wind 70 km/h... dann kommt das Wasser, aber so schwer und so groß, dass es deine Haut umschlägt, dann fiel der Hagel vom Himmel, Menschen paniken und schreien und rennen übereinander.", 'hyp-': "Es beginnt fünf Minuten in der Darstellung ein Wind, der beginnt zu rollen, nachdem ungefähr eine Minute, erreicht der Wind 70 km/h... dann kommt das Wasser, aber so leicht und so klein, dass es deine Haut umschlägt, dann fiel der Hagel vom Himmel, Menschen paniken und schreien und rennen übereinander."}

Target language: German


# Translation French - English

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/gemma/results_gemma_fr_en.csv"

skipped_rows_fr_en = save_results("English", data_fr_en, csv_filename)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Skipping row 3 due to error in data extraction.
Skipping row 5 due to error in data extraction.
Skipping row 6 due to error in data extraction.


,id,langpair,source,hyp+,hyp-
0,0,fr-en,Martelly a intronisé hier un nouveau Conseil é...,Martelly has installed here a new electoral co...,Martelly has installed here a new electoral co...
1,1,fr-en,"est relativement facile en italien, car la pl...","It is relatively easy to learn Italian, as mos...","It is relatively difficult to learn Italian, a..."
2,2,fr-en,Le mysticisme peut être distingué des autres f...,Mysticism can be distinguished from other form...,Mysticism can be distinguished from other form...
3,3,fr-en,Northern avait sollicité une aide en raison de...,NaN,NaN
4,4,fr-en,"Au fil du temps, à mesure que la population s'...",As the population adapts to its new environmen...,As the population adapts to its new environmen...
5,5,fr-en,L'archipel est un État en libre association av...,NaN,NaN
6,6,fr-en,"Bien que Goma soit raisonnablement sûre, toute...",NaN,NaN
7,7,fr-en,Donnez également des exemplaires de la régleme...,You can also give some advice and contacts to ...,You can also give some advice and contacts to ...
8,8,fr-en,Parmi les sports populaires figurent le footba...,"Popular sports include football, basketball, v...","The most popular sports are football, basketba..."
9,9,fr-en,"La société de gestion du groupe, HK Management...","The group management company, HK Management In...","The group management company, HK Management In..."


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


,id,langpair,source,hyp+,hyp-
0,0,fr-en,Martelly a intronisé hier un nouveau Conseil é...,Martelly has installed here a new electoral co...,Martelly has installed here a new electoral co...
1,1,fr-en,"est relativement facile en italien, car la pl...","It is relatively easy to learn Italian, as mos...","It is relatively difficult to learn Italian, a..."
2,2,fr-en,Le mysticisme peut être distingué des autres f...,Mysticism can be distinguished from other form...,Mysticism can be distinguished from other form...
3,3,fr-en,Northern avait sollicité une aide en raison de...,NaN,NaN
4,4,fr-en,"Au fil du temps, à mesure que la population s'...",As the population adapts to its new environmen...,As the population adapts to its new environmen...
5,5,fr-en,L'archipel est un État en libre association av...,NaN,NaN
6,6,fr-en,"Bien que Goma soit raisonnablement sûre, toute...",NaN,NaN
7,7,fr-en,Donnez également des exemplaires de la régleme...,You can also give some advice and contacts to ...,You can also give some advice and contacts to ...
8,8,fr-en,Parmi les sports populaires figurent le footba...,"Popular sports include football, basketball, v...","The most popular sports are football, basketba..."
9,9,fr-en,"La société de gestion du groupe, HK Management...","The group management company, HK Management In...","The group management company, HK Management In..."


...

Skipping row 83 due to error in data extraction.
Skipping row 86 due to error in data extraction.
Skipping row 88 due to error in data extraction.
Skipping row 89 due to error in data extraction.


,id,langpair,source,hyp+,hyp-
0,0,fr-en,Martelly a intronisé hier un nouveau Conseil é...,Martelly has installed here a new electoral co...,Martelly has installed here a new electoral co...
1,1,fr-en,"est relativement facile en italien, car la pl...","It is relatively easy to learn Italian, as mos...","It is relatively difficult to learn Italian, a..."
2,2,fr-en,Le mysticisme peut être distingué des autres f...,Mysticism can be distinguished from other form...,Mysticism can be distinguished from other form...
3,3,fr-en,Northern avait sollicité une aide en raison de...,NaN,NaN
4,4,fr-en,"Au fil du temps, à mesure que la population s'...",As the population adapts to its new environmen...,As the population adapts to its new environmen...
...,...,...,...,...,...
85,85,fr-en,"Windsor est inclus à Augusta, dans le Maine, d...","Windsor is included in Augusta, Maine, in the ...","Windsor is included in Augusta, Maine, in the ..."
86,86,fr-en,Son amiral a rejeté la demande jusqu'à ce qu'i...,NaN,NaN
87,87,fr-en,Sir Thomas Dutton (1er août 1421 -- 23 Septemb...,Sir Thomas Dutton was an English knight of the...,Sir Thomas Dutton was an English knight of the...
88,88,fr-en,"""Son nom, """" Afolabi """", signifie """" Né dans l...",NaN,NaN


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Skipping row 94 due to error in data extraction.
Skipping row 97 due to error in data extraction.


,id,langpair,source,hyp+,hyp-
0,0,fr-en,Martelly a intronisé hier un nouveau Conseil é...,Martelly has installed here a new electoral co...,Martelly has installed here a new electoral co...
1,1,fr-en,"est relativement facile en italien, car la pl...","It is relatively easy to learn Italian, as mos...","It is relatively difficult to learn Italian, a..."
2,2,fr-en,Le mysticisme peut être distingué des autres f...,Mysticism can be distinguished from other form...,Mysticism can be distinguished from other form...
3,3,fr-en,Northern avait sollicité une aide en raison de...,NaN,NaN
4,4,fr-en,"Au fil du temps, à mesure que la population s'...",As the population adapts to its new environmen...,As the population adapts to its new environmen...
...,...,...,...,...,...
95,95,fr-en,"La couleur de capuchon varie, de marron à jaun...","The color of the capuchon changes, from brown ...","The color of the capuchon varies, from brown t..."
96,96,fr-en,La rivière Arieşul Mare est un affluent de la ...,The River Arieşul Mare is a affluent tributary...,The River Arieşul Mare is a major river in Rom...
97,97,fr-en,"Born est né en Allemagne de parents juifs, le ...",NaN,NaN
98,98,fr-en,"À proximité se trouve la rivière Wallowa, un a...","The Wallowa River, a tributary of the Lostine ...",The Wallowa River is located in the northeast ...


In [ ]:
print(skipped_rows_fr_en)
print(len(skipped_rows_fr_en))

[3, 5, 6, 28, 30, 35, 36, 38, 42, 47, 48, 54, 55, 74, 79, 83, 86, 88, 89, 94, 97]
21


In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/gemma/results_gemma_fr_en_fixed.csv"
df_results_fr_en = pd.read_csv(csv_filename)
len(df_results_fr_en)

100

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/gemma/results_gemma_fr_en_fixed.csv"
skipped_rows_fr_en = fix_skipped_rows("English", df_results_fr_en, csv_filename, skipped_rows_fr_en)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


result text  {"hyp+": "Northern had requested assistance due to his exposure to the effects of the subprime crisis in 2007.", "hyp-": "Northern had sought assistance due to his exposure to the effects of the subprime crisis in 2007."}
result text  {"hyp+": "The archipelago is a state in free association with New Zealand, located in Polynesia, in the South Pacific Ocean.", "hyp-": "The archipelago is a state in free association with New Zealand, located in the South Pacific Ocean, but not in Polynesia."}
result text  {"hyp+": "Although Goma is reasonably well-guarded, any visit should be preceded by research to understand the ongoing fighting in North-Kivu.", "hyp-": "Despite Goma's relative security, any visit should be accompanied by extensive research to ensure comprehension of the ongoing conflicts in North-Kivu."}
result text  {"hyp+": "A Dutch player, Ruiter, who was born in Leiden, played for FC Utrecht, FC Den Bosch, Excelsior, RKC Waalwijk, and FC Emmen.", "hyp-": "A Dutch play

In [ ]:
len(skipped_rows_fr_en)
print(skipped_rows_fr_en)

[83, 94, 97]


In [ ]:
fix_skipped_rows_manual("English", df_results_fr_en, skipped_rows_fr_en)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


result text  {'hyp+': 'M. Cave made the highest bid for M. Payne's goods.', 'hyp-': 'M. Cave made the highest bid for M. Payne's goods, but the goods were not sold to him.'}
result text  {'hyp+': "Taylor was active in baseball until his death as a talent scout for the Atlanta Braves and the White Sox of Chicago and Milwaukee.", 'hyp-': "Taylor was active in baseball until his death as a player for the Atlanta Braves and the White Sox of Chicago and Milwaukee."}
result text  {'hyp+': 'Born was born in Germany to Jewish parents, the son of Max Born and the scientist Hedwig Ehrenberg.', 'hyp-': 'Born was born in Germany to Jewish parents, the grandson of Max Born and the scientist Hedwig Ehrenberg.'}


# Translation English - French

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/gemma/results_gemma_en_fr.csv"

skipped_rows_en_fr = save_results("French", data_en_fr, csv_filename)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Skipping row 4 due to error in data extraction.
Skipping row 6 due to error in data extraction.
Skipping row 9 due to error in data extraction.


,id,langpair,source,hyp+,hyp-
0,0,en-fr,"Born in the Croatian capital, , Bobek gained f...","Né en capitale croate, Bobek a gagné la célébr...",Bobek est né à la capitale croate et a gagné l...
1,1,en-fr,Using GPS navigation apps can be the easiest a...,Utiliser les applications de navigation GPS pe...,Utiliser les applications de navigation GPS es...
2,2,en-fr,"Only the effects of catastrophizing, not depre...","Les effets de la catastrophise, mais non la dé...",Les effets de la dépression et la peur étaient...
3,3,en-fr,It can be customized to make it easy to read a...,Le format peut être personnalisé pour facilite...,Le format peut être personnalisé pour facilite...
4,4,en-fr,Stars give off light and heat because of the e...,NaN,NaN
5,5,en-fr,is a species of the family Dromaeosauridae an...,Il est une espèce de la famille des Dromaeosau...,Il est un genre de Dromaeosauridae et est un c...
6,6,en-fr,U.S. President George W. welcomed the announce...,NaN,NaN
7,7,en-fr,The parade of buildings that make the Kong sky...,La parade des bâtiments qui font Scinller le c...,La parade des bâtiments qui font Scinller le c...
8,8,en-fr,The war expenditures of U.S. imperialism in th...,Les dépenses militaires des imperialism améric...,Les dépenses militaires des imperialism améric...
9,9,en-fr,With roots in Barcelona's Running Tours Barcel...,NaN,NaN


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Skipping row 10 due to error in data extraction.
Skipping row 11 due to error in data extraction.


...

Skipping row 82 due to error in data extraction.
Skipping row 85 due to error in data extraction.
Skipping row 86 due to error in data extraction.
Skipping row 87 due to error in data extraction.


,id,langpair,source,hyp+,hyp-
0,0,en-fr,"Born in the Croatian capital, , Bobek gained f...","Né en capitale croate, Bobek a gagné la célébr...",Bobek est né à la capitale croate et a gagné l...
1,1,en-fr,Using GPS navigation apps can be the easiest a...,Utiliser les applications de navigation GPS pe...,Utiliser les applications de navigation GPS es...
2,2,en-fr,"Only the effects of catastrophizing, not depre...","Les effets de la catastrophise, mais non la dé...",Les effets de la dépression et la peur étaient...
3,3,en-fr,It can be customized to make it easy to read a...,Le format peut être personnalisé pour facilite...,Le format peut être personnalisé pour facilite...
4,4,en-fr,Stars give off light and heat because of the e...,NaN,NaN
...,...,...,...,...,...
85,85,en-fr,"Mats Wilander defeats Anders Järryd, 6 -- 4, 3...",NaN,NaN
86,86,en-fr,The final line of the third verse was changed ...,NaN,NaN
87,87,en-fr,"On April 4, 1892, the second ship, ``Missouri,...",NaN,NaN
88,88,en-fr,He was selected by the Philadelphia 76ers in t...,Il a été sélectionné par les Philadelphia 76er...,Il a été sélectionné par les Philadelphia 76er...


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Skipping row 91 due to error in data extraction.
Skipping row 92 due to error in data extraction.
Skipping row 99 due to error in data extraction.


,id,langpair,source,hyp+,hyp-
0,0,en-fr,"Born in the Croatian capital, , Bobek gained f...","Né en capitale croate, Bobek a gagné la célébr...",Bobek est né à la capitale croate et a gagné l...
1,1,en-fr,Using GPS navigation apps can be the easiest a...,Utiliser les applications de navigation GPS pe...,Utiliser les applications de navigation GPS es...
2,2,en-fr,"Only the effects of catastrophizing, not depre...","Les effets de la catastrophise, mais non la dé...",Les effets de la dépression et la peur étaient...
3,3,en-fr,It can be customized to make it easy to read a...,Le format peut être personnalisé pour facilite...,Le format peut être personnalisé pour facilite...
4,4,en-fr,Stars give off light and heat because of the e...,NaN,NaN
...,...,...,...,...,...
95,95,en-fr,"William Llewelyn Williams, better known as Lle...","William Llewelyn Williams, mieux connu sous le...","William Llewelyn Williams, mieux connu sous le..."
96,96,en-fr,"Flavia Alejandra Gleske Fajin, better known as...",Flavia Gleske est une actrice et un modèle Vén...,Flavia Gleske Fajin est une actrice et un modè...
97,97,en-fr,"In the 1930s and 1940s, he was elected four ti...","En les années 1930 et 1940, il a été élu quatr...","En les années 1930 et 1940, il a été élu plusi..."
98,98,en-fr,In 1924 he was an invited spokesperson for the...,"En 1924, il était un orateur invité pour la So...","En 1924, il était un orateur invité pour la So..."


In [ ]:
print(skipped_rows_en_fr)
print(len(skipped_rows_en_fr))

[4, 6, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 24, 26, 29, 30, 31, 32, 37, 41, 42, 44, 45, 48, 49, 50, 51, 52, 57, 58, 59, 60, 61, 62, 69, 70, 72, 76, 77, 78, 80, 82, 85, 86, 87, 91, 92, 99]
51


In [23]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/gemma/results_gemma_en_fr_fixed.csv"
df_results_en_fr = pd.read_csv(csv_filename)
len(df_results_en_fr)

100

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/gemma/results_gemma_en_fr_fixed.csv"
skipped_rows_en_fr = fix_skipped_rows("French", df_results_en_fr, csv_filename, skipped_rows_en_fr)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


result text  {"hyp+": "Les étoiles émettent de la lumière et de la chaleur en raison de l'énergie créée lorsque les atomes sont fusionnés pour forment des éléments plus lourds.", "hyp-": "Les étoiles émettent de la lumière et de la chaleur en raison de la chaleur créée lorsque les atomes sont fusionnés pour forment des éléments plus lourds."}
result text  {"hyp+": "Le Président des États-Unis, George W., a reçu avec enthousiasme l'annonce.", "hyp-": "Le Président des États-Unis, George W., a accepté l'annonce."}
result text  {"hyp+": "Les tourings Running sont établies à Barcelona et Copenhagen, et ont été rapidement joined par Running Tours basée à Prague et d'autres.", "hyp-": "Les tourings Running sont établies à Barcelona et Copenhagen, et ont été rapidement joined par Running Tours basée à Prague et d'autres tourings Running."}
result text  {"hyp+": "Couchsurfing a fondé en janvier 2004 après que le programmeur informatique Casey Fenton a trouvé un vol abordable en Islande mais n'

In [ ]:
len(skipped_rows_en_fr)
print(skipped_rows_en_fr)

[14, 16, 42, 45, 57, 86, 92]


In [ ]:
fix_skipped_rows_manual("French", df_results_en_fr, skipped_rows_en_fr)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


result text  {'hyp+': 'Le nombre de personnes qui ont confirmé la découverte a changé, mais le message a été annoncé le mardi. ', 'hyp-' : 'Le nombre de personnes qui ont confirmé la découverte est resté secret, mais le message a été annoncé le mardi. '}
result text  {'hyp+': "The Onion a gagné en popularité comme un empire de parodie de nouvelles, avec une édition en print, un site web qui a enregistré 5 millions de visiteurs uniques en octobre, des publicités personnelles, un réseau de nouvelles 24 heures, des podcasts et un atlas du monde récemment lancé appelé Notre Monde Ignorant.", 'hyp-': "The Onion a gagné en popularité comme un empire de parodie de nouvelles, avec une édition en print, un site web qui a enregistré 5 millions de visiteurs uniques en octobre, des publicités personnelles, un réseau de nouvelles 24 heures, des podcasts et un atlas du monde récemment lancé appelé Notre Monde Ignorant."}
result text  {'hyp+': 'Il a également donné 34 000 $ à Ted Cruz et 7 000 $ aux 

In [24]:
skipped_rows_en_fr = [14, 16, 42, 45, 57, 86, 92]
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/gemma/results_gemma_en_fr_fixed2.csv"
skipped_rows_en_fr = fix_skipped_rows("French", df_results_en_fr, csv_filename, skipped_rows_en_fr)

result text  {"hyp+": "Un certain nombre de personnes ont confirmé la découverte à l'aide de différents hardware et logiciels, à la début de février, et l'annonce a été faite le mardi.", "hyp-": "La découverte a été confirmée par un grand nombre de personnes, utilisant différents hardware et logiciels, au début de février, et l'annonce a été faite le mardi."}
result text  {"hyp+": "L'émergence de The Onion a créé un empire de parodie d'information, avec une édition papier, un site web qui a rassemblé 5 millions de visiteurs uniques en octobre, des annonces personnelles, un réseau de nouvelles 24 heures, des podcasts et un atlas du monde récemment lancé appelé Notre monde imprud. ', 'hyp-': 'Depuis sa création, The Onion est devenue un empire de parodie d'information, avec une édition papier, un site web qui a rassemblé 5 millions de visiteurs uniques en octobre, des annonces personnelles, un réseau de nouvelles 24 heures, des podcasts et un atlas du monde récemment lancé appelé Notre m

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


result text  {"hyp+": "Le quatème était causé par la démissionciation de Thomas Kittera quelque temps après mai 1826, rempli par Joseph Hemphill.", "hyp-": "Le quatème était causé par la démissionciation de Thomas Kittera quelque temps après mai 1826, rempli par Joseph Hemphill. Mais il n\'était pas rempli.}
Skipping row 92 due to error in data extraction.


### Basic prompt with examples (Few-Shot):

In [ ]:
# or for loop
prompt = "Given the src below, generate a translation hypothesis hyp+ that is supported by src and a second translation hyp- that is not supported by src. Here are some examples:\n"

x = 0
for ex in data_de_en:
  if x < 4:
    x += 1
  else:
    x += 1
    if x == 5:
      for i in range(4):
        if trial_ds_de_en[i]["label"] == "hyp1":
          prompt += f'\nExample {i+1}:\nsrc: {trial_ds_de_en[i]["source"]}\nhyp+: {trial_ds_de_en[i]["hyp2"]}\nhyp-: {trial_ds_de_en[i]["hyp1"]}'
        else:
          prompt += f'\nExample {i+1}:\nsrc: {trial_ds_de_en[i]["source"]}\nhyp+: {trial_ds_de_en[i]["hyp1"]}\nhyp-: {trial_ds_de_en[i]["hyp2"]}'
      prompt += "\nsource: " + ex['source']
      prompt += "\ntarget language: english"
      x+=1
    else:
      break
print(prompt)


In [ ]:
input_ids = tokenizer(prompt, return_tensors="pt")

outputs = model.generate(**input_ids, max_new_tokens=200)
print(tokenizer.decode(outputs[0]))

### Prompt with role and task

In [ ]:
# or for loop
prompt = "Imagine you are a hallucination generator. Given the src below, generate a translation hypothesis hyp+ that is supported by src and a second translation hyp- that is not supported by src. Here are some examples:\n"

x = 0
for ex in data_de_en:
  if x < 4:
    x += 1
  else:
    x += 1
    if x == 5:
      for i in range(4):
        if trial_ds_de_en[i]["label"] == "hyp1":
          prompt += f'\nExample {i+1}:\nsrc: {trial_ds_de_en[i]["source"]}\nhyp+: {trial_ds_de_en[i]["hyp2"]}\nhyp-: {trial_ds_de_en[i]["hyp1"]}'
        else:
          prompt += f'\nExample {i+1}:\nsrc: {trial_ds_de_en[i]["source"]}\nhyp+: {trial_ds_de_en[i]["hyp1"]}\nhyp-: {trial_ds_de_en[i]["hyp2"]}'
      prompt += "\nsource: " + ex['source']
      prompt += "\ntarget language: english"
      x+=1
    else:
      break
print(prompt)


In [ ]:
input_ids = tokenizer(prompt, return_tensors="pt")

outputs = model.generate(**input_ids, max_new_tokens=200)
print(tokenizer.decode(outputs[0]))

## Prompt with explanation

In [ ]:
# or for loop
prompt = "Given the src below, generate a translation hypothesis hyp+ that is supported by src and a second translation hyp- that is not supported by src. Here are some examples:\n"
prompt += f'\nExample 1:\nsrc: In der Phase marschieren Wanderameisen bei Nacht und machen tagsüber Rast.\nhyp+: In the nomadic phase, wandering ants march at night and rest during the day.\nhyp-: In the nomadic phase, wandering ants march at night and rest during the day.'
prompt += "\nHyp- is not supported by src, since the 'phase' is defined as 'nomadic', which cannot be found in the src."
prompt += "\nExample 2:\nsrc: Nachdem sich der Frontsänger Steven Tyler während eines Auftrittes am 5. August bei einem Sturz von der Bühne verletzte, haben sie die Tour abgebrochen.\nhyp+: After front singer Steven Tyler injured himself in a fall from the stage during a performance on Aug. 5, they canceled the tour.\nhyp-: After front singer Steven Tyler injured himself in a fall from the stage during a performance on December 5, they canceled the tour."
prompt += "\nHyp- is not supported by src, since the date is incorrect"
prompt += "\nsrc: Ein Minenabwehrschiff der Avenger-Klasse, das Schiff war auf dem Weg in Palawan.\ntarget language: english"
print(prompt)

In [ ]:
input_ids = tokenizer(prompt, return_tensors="pt")

outputs = model.generate(**input_ids, max_new_tokens=200)
print(tokenizer.decode(outputs[0]))

In [ ]:
prompt = f'''Given the src="Ein Minenabwehrschiff der Avenger-Klasse, das Schiff war auf dem Weg in Palawan.", generate a translation hypothesis hyp+ that is supported by src and a second translation hyp- that is not supported by src and a hallucination. Target language: english\n'''
print(prompt)

In [ ]:
input_ids = tokenizer(prompt, return_tensors="pt")

outputs = model.generate(**input_ids, max_new_tokens=200)
print(tokenizer.decode(outputs[0]))